In [1]:
!pip install -qU \
  langchain-core \
  langchain-google-genai \
  langchain-community \
  scikit-image


In [2]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

Enter API key for Google Gemini: ········


In [3]:
from langchain_google_genai import GoogleGenerativeAI


llm = GoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.0) 

## Conversation Buffer Memory

In [4]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

/tmp/ipykernel_5240/1448044083.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)


In [5]:
memory.save_context(
    {"input": "Hi, my names in James"},
    {"output": "Hey James, what's up? I'm an AI model called Zeta."}
)
memory.save_context(
    {"input": "I am researching different types of conversational memory."},
    {"output": "That's interesting, what are some examples?"}
)
memory.save_context(
    {"input": "I am researching ConversationBufferMemory and ConversationBufferWindowMemory"},
    {"output": "That's interesting, what's thed difference"}
)
memory.save_context(
    {"input": "Buffer memory just store the enitre converation, right?"},
    {"output": "Thats makes sense, what about ConversationBufferWindowMemory"}
)
memory.save_context(
    {"input": "Buffer window memory stores last k messages, dropping the rest."},
    {"output": "Very cool!"}
)


In [6]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi, my names in James', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey James, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I am researching different types of conversational memory.', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I am researching ConversationBufferMemory and ConversationBufferWindowMemory', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's thed difference", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just store the enitre converation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Thats makes sense, what about ConversationBufferWindowMemory', additional_kwargs={}, response_metadata={}),
  HumanMessage(conte

In [7]:
from langchain.chains import ConversationChain

chain = ConversationChain(
    llm = llm,
    memory = memory,
    verbose = True
)


/tmp/ipykernel_5240/2943370833.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  chain = ConversationChain(


In [8]:
chain.invoke({"input": "What is my name again?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='Hi, my names in James', additional_kwargs={}, response_metadata={}), AIMessage(content="Hey James, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}), HumanMessage(content='I am researching different types of conversational memory.', additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}), HumanMessage(content='I am researching ConversationBufferMemory and ConversationBufferWindowMemory', additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what's thed difference", additional_kwargs={}

{'input': 'What is my name again?',
 'history': [HumanMessage(content='Hi, my names in James', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey James, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I am researching different types of conversational memory.', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I am researching ConversationBufferMemory and ConversationBufferWindowMemory', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's thed difference", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just store the enitre converation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Thats makes sense, what about ConversationBufferWindowMemory', additional_kwargs={}, respons

## ConversationBufferMemory with RunnableWithMessageHistory

In [9]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    ChatPromptTemplate
)

system_prompt = "You are a helpful assistant called Zeta."

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{query}")   
])


In [10]:
pipeline = (
    { 
        "query": lambda x: x["query"],
        "history": lambda x: x["history"]
    }
    | prompt_template
    | llm
)

In [11]:
from langchain_core.chat_history import InMemoryChatMessageHistory

chat_map = {}
def get_chat_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in chat_map:
        # if session ID doesn't exist, create a new chat history
        chat_map[session_id] = InMemoryChatMessageHistory()
    return chat_map[session_id]

In [12]:
from langchain_core.runnables.history import RunnableWithMessageHistory

pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history"
)

In [13]:
pipeline_with_history.invoke(
    {"query": "Hi, my name is James"},
    config={"session_id": "id_123"}
)

"Hello James, it's nice to meet you. My name is Zeta. How can I help you today?"

In [14]:
pipeline_with_history.invoke(
    {"query": "What is my name again?"},
    config={"session_id": "id_123"}
)

'Your name is James.'

## ConversationBufferWindowMemory

In [15]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=4, return_messages=True)

/tmp/ipykernel_5240/3216785012.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=4, return_messages=True)


In [16]:
memory.save_context(
    {"input": "Hi, my names in James"},
    {"output": "Hey James, what's up? I'm an AI model called Zeta."}
)
memory.save_context(
    {"input": "I am researching different types of conversational memory."},
    {"output": "That's interesting, what are some examples?"}
)
memory.save_context(
    {"input": "I am researching ConversationBufferMemory and ConversationBufferWindowMemory"},
    {"output": "That's interesting, what's thed difference"}
)
memory.save_context(
    {"input": "Buffer memory just store the enitre converation, right?"},
    {"output": "Thats makes sense, what about ConversationBufferWindowMemory"}
)
memory.save_context(
    {"input": "Buffer window memory stores last k messages, dropping the rest."},
    {"output": "Very cool!"}
)

In [17]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='I am researching different types of conversational memory.', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I am researching ConversationBufferMemory and ConversationBufferWindowMemory', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's thed difference", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just store the enitre converation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Thats makes sense, what about ConversationBufferWindowMemory', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer window memory stores last k messages, dropping the rest.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Very cool!', additional_kwargs={}, response_metadata={})]}

In [18]:
chain = ConversationChain(
    llm = llm,
    memory = memory,
    verbose = True
)

In [19]:
chain.invoke({"input": "What is my name again"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='I am researching different types of conversational memory.', additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}), HumanMessage(content='I am researching ConversationBufferMemory and ConversationBufferWindowMemory', additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what's thed difference", additional_kwargs={}, response_metadata={}), HumanMessage(content='Buffer memory just store the enitre converation, right?', additional_kwargs={}, response_metadata={}), AIMessage(content='Thats makes sense, what about Conversat

{'input': 'What is my name again',
 'history': [HumanMessage(content='I am researching different types of conversational memory.', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I am researching ConversationBufferMemory and ConversationBufferWindowMemory', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's thed difference", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just store the enitre converation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Thats makes sense, what about ConversationBufferWindowMemory', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer window memory stores last k messages, dropping the rest.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Very cool!', additional_kwargs={}, respon

In [20]:
from pydantic import BaseModel, Field
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage

class BufferWindowMessageHistory(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)
    k: int = Field(default_factory=int)
    
    
    def __init__(self, k:int):
        super().__init__(k=k)
        print(f"Initializing BufferWindowMessageHistory with k={k}")
        
    
    def add_messages(self, messages: list[BaseMessage]) -> None:
        """
        Add messages to the history, removing any messages beyond
        the last `k` messages
        """
        
        self.messages.extend(messages)
        self.messages = self.messages[-self.k:]
        
        
    def clear(self) -> None:
        """
        clear the history
        """
        self.messages = []       
        
        

In [21]:
chat_map = {}
def get_chat_window_history(session_id: str, k: int = 4) -> BufferWindowMessageHistory:
    if session_id not in chat_map:
        chat_map[session_id] = BufferWindowMessageHistory(k=k)
    
    return chat_map[session_id]

In [22]:
from langchain_core.runnables import ConfigurableFieldSpec

pipeline_cwbm = (
    { 
        "query": lambda x: x["query"],
        "history": lambda x: x["history"]
    }
    | prompt_template
    | llm
)
 

pipeline_with_history_cwbm = RunnableWithMessageHistory(
    pipeline_cwbm,
    get_session_history = get_chat_window_history,
    input_messages_key = "query",
    history_messages_key = "history",
    history_factory_config = [
        ConfigurableFieldSpec(
            id = "session_id",
            annotation = str,
            name = "Session ID",
            description = "The session ID to use for the chat history",
            default = "id_default"
        ),
        ConfigurableFieldSpec(
            id = "k",
            annotation = int,
            name = "k",
            description = "The number of messages to keep in the history",
            default = 4
        )
    ]
    
)

In [23]:
pipeline_with_history_cwbm.invoke(
    {
        "query": "my name is James"  
    },
    config = {
        "configurable": {
            "session_id" : "id_k3",
            "k" : 3
        }
    }

)

Initializing BufferWindowMessageHistory with k=3


"Okay, James. It's nice to meet you. How can I help you today?"

In [24]:
pipeline_with_history_cwbm.invoke(
    {
        "query": "I am learning langchain"  
    },
    config = {
        "configurable": {
            "session_id" : "id_k3",
            "k" : 3
        }
    }

)

"AI: That's great, James! Langchain is a powerful framework. What are you hoping to learn about it? Are you just starting out, or do you have some specific questions or goals in mind?\n\nFor example, are you interested in:\n\n*   **Understanding the core concepts?** (e.g., Chains, LLMs, Prompts, Memory, Agents)\n*   **Building a specific type of application?** (e.g., Question Answering, Chatbots, Document Summarization)\n*   **Working with a particular LLM?** (e.g., OpenAI, Hugging Face, Llama 2)\n*   **Troubleshooting a specific issue?**\n\nThe more information you give me, the better I can assist you. Let me know what you'd like to focus on!"

In [25]:
pipeline_with_history_cwbm.invoke(
    {
        "query": "My plan is to learn Langgraph and Google A2A, which one should I go"  
    },
    config = {
        "configurable": {
            "session_id" : "id_k3",
            "k" : 3
        }
    }

)

"AI: Okay, James. LangGraph and Google A2A (Agent-to-Agent communication) are both interesting and relatively advanced topics within the Langchain ecosystem. Choosing which to learn first depends on your goals and current understanding.\n\nHere's a breakdown to help you decide:\n\n**LangGraph:**\n\n*   **What it is:** LangGraph is a framework built on top of Langchain that allows you to create more complex, stateful, and cyclical conversational flows. Think of it as a way to orchestrate multiple Langchain components (LLMs, tools, etc.) in a graph-like structure.  It's designed for building applications that require more than just a simple linear chain of operations.\n*   **Use Cases:**  Good for building complex agents that need to reason over multiple steps, handle feedback loops, or manage state across interactions. Examples include:\n    *   Multi-agent collaboration systems\n    *   Complex decision-making processes\n    *   Applications requiring iterative refinement of results\n*

In [26]:
chat_map["id_k3"].messages[2]

AIMessage(content="AI: Okay, James. LangGraph and Google A2A (Agent-to-Agent communication) are both interesting and relatively advanced topics within the Langchain ecosystem. Choosing which to learn first depends on your goals and current understanding.\n\nHere's a breakdown to help you decide:\n\n**LangGraph:**\n\n*   **What it is:** LangGraph is a framework built on top of Langchain that allows you to create more complex, stateful, and cyclical conversational flows. Think of it as a way to orchestrate multiple Langchain components (LLMs, tools, etc.) in a graph-like structure.  It's designed for building applications that require more than just a simple linear chain of operations.\n*   **Use Cases:**  Good for building complex agents that need to reason over multiple steps, handle feedback loops, or manage state across interactions. Examples include:\n    *   Multi-agent collaboration systems\n    *   Complex decision-making processes\n    *   Applications requiring iterative refine

## Converasation Summary Memory

In [27]:
from langchain.memory import ConversationSummaryMemory

summary_memory = ConversationSummaryMemory(llm = llm)

/tmp/ipykernel_5240/3825381965.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  summary_memory = ConversationSummaryMemory(llm = llm)


In [28]:
summary_chain = ConversationChain(
    llm = llm,
    memory = summary_memory,
    verbose = True
)

In [29]:
summary_chain.invoke({"input": "hello there my name in Muhammad"})
summary_chain.invoke({"input": "I am learning chain and exploring how it can be use"})
summary_chain.invoke({"input": "I am looking into ConversationBufferMemory and ConversationWindowBufferMemory"})
summary_chain.invoke({"input": "BufferMemory stores the entire conversation"})
summary_chain.invoke({"input": "BufferWindowMemory stores last k messages and dropped previous messages"})





> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hello there my name in Muhammad
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human introduces himself as Muhammad. The AI greets Muhammad, explains it doesn't have a name in the human sense but can be called AI, and describes itself as a large language model trained by Google, capable of communicating and generating human-like text. It then asks how it can help 

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 22
}
].


ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 20
}
]

In [30]:
summary_chain.invoke({"input": "What is my name"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human introduces himself as Muhammad. The AI greets Muhammad, explains it doesn't have a name in the human sense but can be called AI, and describes itself as a large language model trained by Google, capable of communicating and generating human-like text. It then asks how it can help Muhammad. Muhammad says he is learning about "chain" and exploring its uses. The AI confirms that Muhammad is referring to chaining operations or functions, and asks for clarification on the specific area of interest, providing examples like functional programming chains, data pipeline chains, blockchain technology, and chains of thought in AI, offering to provide specific informatio

{'input': 'What is my name',
 'history': 'The human introduces himself as Muhammad. The AI greets Muhammad, explains it doesn\'t have a name in the human sense but can be called AI, and describes itself as a large language model trained by Google, capable of communicating and generating human-like text. It then asks how it can help Muhammad. Muhammad says he is learning about "chain" and exploring its uses. The AI confirms that Muhammad is referring to chaining operations or functions, and asks for clarification on the specific area of interest, providing examples like functional programming chains, data pipeline chains, blockchain technology, and chains of thought in AI, offering to provide specific information, examples, and resources based on Muhammad\'s interest. Muhammad specifies he is looking into ConversationBufferMemory and ConversationWindowBufferMemory. The AI confirms these are useful components in conversational AI for chatbots and agents needing to remember past interacti

## Conversation Summary Memory with Runnable Message History

In [71]:
from langchain_core.messages import SystemMessage

In [72]:
class ConversationSummaryMessageHistory(BaseChatMessageHistory, BaseModel): 
    messages: list[BaseMessage] = Field(default_factory=list)
    llm: GoogleGenerativeAI = Field(default_factory=GoogleGenerativeAI)
    
    def __init__(self, llm:GoogleGenerativeAI):
        super().__init__(llm=llm)
    
    def add_messages(self, messages: list[BaseMessage]) -> None:
        """Add messages to the history
        """
        self.messages.extend(messages)
        # construct the summary chat messages
        summary_prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(
                "Given the existing conversation summary and the new messages, "
                "generate a new summary of the conversation. Ensuring to maintain "
                "as much relevant information as possible."
            ),
            HumanMessagePromptTemplate.from_template(
                "Existing conversation summary:\n{existing_summary}\n\n"
                "New messages:\n{messages}"
            )
        ])
        # format the messages and invoke the LLM
        new_summary = self.llm.invoke(
            summary_prompt.format_messages(
                existing_summary=self.messages,
                messages=[x.content for x in messages]
            )
        )
        # replace the existing history with a single system summary message
        self.messages = [SystemMessage(content=new_summary)]

    def clear(self) -> None:
        """Clear the history."""
        self.messages = []
    

In [77]:
summary_chat_map = {}

def get_summary_chat_history(session_id: str, llm: GoogleGenerativeAI) -> ConversationSummaryMessageHistory:
    if session_id not in summary_chat_map:
        summary_chat_map[session_id] = ConversationSummaryMessageHistory(llm = llm)
    
    return summary_chat_map[session_id]

In [78]:
pipeline_csmh = (
        { 
        "query": lambda x: x["query"],
        "history": lambda x: x["history"]
        }
    | prompt_template
    | llm
)

pipeline_with_summary_history = RunnableWithMessageHistory(
    pipeline_csmh,
    get_session_history = get_summary_chat_history,
    input_messages_key = "query",
    history_messages_key = "history",
    history_factory_config = [
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="The session ID to use for the chat history",
            default="id_default",
        ),
        ConfigurableFieldSpec(
            id="llm",
            annotation=GoogleGenerativeAI,
            name="LLM",
            description="The LLM to use for the conversation summary",
            default=llm,
        )
        
    ]
)

In [79]:
session_id = "csbm_123"

pipeline_with_summary_history.invoke(
    {
        "query": "Hi, my name is Muhammad"
    },
    config={
        "session_id": session_id, 
        "llm": llm
    }
)

"Hello Muhammad, it's nice to meet you! My name is Zeta. How can I help you today?"

In [80]:
summary_chat_map[session_id].messages

[SystemMessage(content='The conversation began with Muhammad introducing himself. Zeta, the AI, responded, introducing herself and offering assistance.', additional_kwargs={}, response_metadata={})]

In [81]:
pipeline_with_summary_history.invoke(
    {
        "query": "I'm researching different type of conversational memory"
    },
    config={
        "session_id": session_id, 
        "llm": llm
    }
)

"Okay, Muhammad. I understand you're researching different types of conversational memory. That's a fascinating area! To help me understand what you're looking for, could you tell me a bit more about what aspects you're interested in? For example, are you focusing on:\n\n*   **Specific types of memory architectures?** (e.g., episodic memory, semantic memory, working memory, long-term memory, etc.)\n*   **Different techniques for implementing conversational memory in AI systems?** (e.g., using embeddings, knowledge graphs, attention mechanisms, etc.)\n*   **The challenges and limitations of current conversational memory approaches?** (e.g., dealing with context length, maintaining consistency, handling ambiguity, etc.)\n*   **Applications of conversational memory in specific domains?** (e.g., customer service, education, healthcare, etc.)\n\nThe more information you can give me, the better I can assist you in your research. I can provide information, suggest resources, and help you brai

In [83]:
for msg in [
    "I have been looking at ConversationBufferMemory and ConversationBufferWindowMemory.",
    "Buffer memory just stores the entire conversation",
    "Buffer window memory stores the last k messages, dropping the rest."
]:
    pipeline_with_summary_history.invoke(
        {"query": msg},
        config={"session_id": session_id, "llm": llm}
    )

In [84]:
summary_chat_map[session_id].messages

[SystemMessage(content='The conversation began with Muhammad introducing himself to Zeta, the AI, stating he is researching different types of conversational memory. Zeta offered assistance and asked for clarification on the specific aspects of conversational memory Muhammad is interested in, such as memory architectures, implementation techniques, challenges, or applications in specific domains. Muhammad stated he has been looking at ConversationBufferMemory and ConversationBufferWindowMemory. Zeta responded by confirming these are good starting points. Muhammad then correctly stated that Buffer memory stores the entire conversation. Zeta affirmed this, elaborating on ConversationBufferMemory as the simplest approach, storing the entire conversation history as a single string, making it easy to access the full context. Muhammad then correctly stated that Buffer window memory stores the last k messages, dropping the rest. Zeta confirmed this, explaining that ConversationBufferWindowMem

In [85]:
    pipeline_with_summary_history.invoke(
        {"query": "what is my name again"},
        config={"session_id": session_id, "llm": llm}
    )

'Your name is Muhammad.'